# torture

In [ ]:
import sqlite3
import urllib.error
import ssl
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

con = sqlite3.connect("helpme.sqlite3")
cur = con.cursor()
cur.executescript("""create table if not exists Origins (site text unique);
                create table if not exists Pages (id integer primary key, page text unique, html TEXT, error text, old_rank real, new_rank real);
                create table if not exists mtm (from_page integer, to_page integer, unique(from_page, to_page))""")

while True:
    site = input("start page?: ")
    if site == "done":
        break
    else:pass
    cur.execute("select page from Pages where page = ?", (site,))
    if cur.fetchone() is not None:
        print("already in database")
        continue
    else:pass
    if len(site)<1:
        cur.execute('SELECT id, page, html FROM Pages ORDER BY RANDOM() LIMIT 1')
        site2 = cur.fetchone()
        try:
            site = site2[1][:re.search(r"\.[a-z]{2,3}(/)", site2[1]).span()[1] - 1]
        except:
            print("couldn't extract domain")
            continue
        html = BeautifulSoup(site2[2], "html.parser")
        site2 = site2[0]
    else:
        try:
            site1 = urlopen(site, context=ctx)
            html = BeautifulSoup(site1.read(), "html.parser")
            cur.execute('INSERT OR IGNORE INTO Pages (page, html, error) VALUES ( ?, ?, ?)',
                        (site, html.prettify(), site1.getcode()))
            cur.execute('SELECT id, page FROM Pages where page = ?', (site,))
            site2 = cur.fetchone()[0]
            site = site[:re.search(r"\.[a-z]{2,3}(/)", site).span()[1] - 1]
            cur.execute('INSERT OR IGNORE INTO Origins (site) VALUES ( ? )', (site,))
        except:
            print("smth fucked up")
            continue
    for tag in html('a'):
        try:
            href = tag.get('href', None)
            if site not in href:
                continue
            else:
                ipos = href.find('#')
                if ipos > 1:
                    href = href[:ipos]
                if href.endswith('.png') or href.endswith('.jpg') or href.endswith('.gif'):
                    continue
                if href.endswith('/'):
                    href = href[:-1]
                html = BeautifulSoup(urlopen(href, context=ctx).read(), "html.parser")
                cur.execute('INSERT OR IGNORE INTO Pages (page, html, error) VALUES ( ?, ?, ?)', (href, html.prettify(), urlopen(href, context=ctx).getcode()))
                cur.execute("select id from Pages where page = ?", (href,))
                site3 = cur.fetchone()[0]
                cur.execute('INSERT OR IGNORE INTO mtm (from_page, to_page) VALUES ( ?, ?)',
                            (site2, site3))
        except:
            continue
    con.commit()